## 1. Environment Setup and Installation

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"Number of GPUs: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"    Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.1f} GB")

In [ ]:
# Install required packages
!pip install -q cupy-cuda11x  # For CUDA 11.x (adjust if using CUDA 12.x)
!pip install -q scikit-learn matplotlib seaborn pillow tqdm pandas

In [ ]:
# Import libraries
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from datetime import datetime
import json
from typing import Dict, List, Tuple

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# Scientific computing
from scipy import signal
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

# GPU acceleration
try:
    import cupy as cp
    CUPY_AVAILABLE = True
    GPU_AVAILABLE = cp.cuda.is_available()
    print("    CuPy installed and GPU available for connectivity calculations")
except ImportError:
    cp = np
    CUPY_AVAILABLE = False
    GPU_AVAILABLE = False
    print("⚠ CuPy not available. Using CPU for connectivity calculations.")

print(f"\nSetup complete!")
print(f"PyTorch device: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")
print(f"Connectivity GPU: {'Enabled' if GPU_AVAILABLE else 'Disabled (CPU)'}")

## 2. Configuration

In [ ]:
# Paths - Adjust these based on your Kaggle dataset structure
DEAP_PATH = '/kaggle/input/deap-dataset/data_preprocessed_python'  # Update this path
OUTPUT_DIR = '/kaggle/working/outputs'
CACHE_DIR = os.path.join(OUTPUT_DIR, 'cache')
MODEL_DIR = os.path.join(OUTPUT_DIR, 'models')
RESULTS_DIR = os.path.join(OUTPUT_DIR, 'results')
FIGURES_DIR = os.path.join(OUTPUT_DIR, 'figures')

# Create directories
for dir_path in [OUTPUT_DIR, CACHE_DIR, MODEL_DIR, RESULTS_DIR, FIGURES_DIR]:
    os.makedirs(dir_path, exist_ok=True)

# EEG Configuration
class Config:
    # DEAP Dataset
    NUM_SUBJECTS = 32
    NUM_VIDEOS = 40
    NUM_CHANNELS = 32
    SAMPLING_RATE = 128  # Hz
    VIDEO_LENGTH = 60  # seconds
    VALENCE_THRESHOLD = 4.5
    AROUSAL_THRESHOLD = 4.5
    
    # Windowing
    WINDOW_LENGTH = 5  # seconds
    OVERLAP_PERCENT = 0.8
    NUM_CONSECUTIVE_WINDOWS = 3
    
    # Effective Connectivity
    TE_NUM_NEIGHBORS = 4
    TE_EMBEDDING_DIM = 3
    TE_TIME_DELAY = 10
    AR_MODEL_ORDER = 10
    EC_IMAGE_SIZE = 32
    FUSED_IMAGE_SIZE = 96  # 32*3
    USE_GPU_CONNECTIVITY = GPU_AVAILABLE
    
    # Emotion Classes
    NUM_CLASSES = 4
    CLASS_NAMES = ['Q1', 'Q2', 'Q3', 'Q4']
    
    # Training
    LEARNING_RATE = 0.0004
    MAX_EPOCHS = 40
    BATCH_SIZE = 32
    NUM_WORKERS = 2  # Kaggle has limited CPU cores
    
    # Multi-GPU
    USE_MULTI_GPU = torch.cuda.device_count() > 1
    
    # Model selection
    CNN_MODEL = 'ResNet50'  # Options: ResNet50, InceptionV3, DenseNet201, EfficientNetB0
    
config = Config()

print(f"Configuration:")
print(f"  Dataset: DEAP ({config.NUM_SUBJECTS} subjects, {config.NUM_VIDEOS} videos)")
print(f"  Multi-GPU: {config.USE_MULTI_GPU} ({torch.cuda.device_count()} GPUs)")
print(f"  GPU Connectivity: {config.USE_GPU_CONNECTIVITY}")
print(f"  Model: {config.CNN_MODEL}")
print(f"  Batch Size: {config.BATCH_SIZE}")
print(f"  Max Epochs: {config.MAX_EPOCHS}")

## 3. Data Loader

In [ ]:
class DEAPDataLoader:
    """Load and process DEAP dataset"""
    
    def __init__(self, data_path=DEAP_PATH):
        self.data_path = data_path
        self.config = config
        
    def load_subject_data(self, subject_id: int) -> Dict:
        """Load data for a single subject (1-32)"""
        filename = f"s{subject_id:02d}.dat"
        filepath = os.path.join(self.data_path, filename)
        
        if not os.path.exists(filepath):
            raise FileNotFoundError(f"Subject data not found: {filepath}")
        
        # Load pickle file
        with open(filepath, 'rb') as f:
            subject_data = pickle.load(f, encoding='latin1')
        
        # Extract EEG data (32 channels) and labels
        eeg_data = subject_data['data'][:, :32, :]
        labels = subject_data['labels']
        
        # Skip 3s baseline, take 60s video
        baseline_samples = 3 * self.config.SAMPLING_RATE
        video_samples = 60 * self.config.SAMPLING_RATE
        eeg_data = eeg_data[:, :, baseline_samples:baseline_samples + video_samples]
        
        # Convert to emotion classes
        emotion_classes = self._labels_to_classes(labels)
        
        return {
            'eeg_data': eeg_data,
            'labels': labels,
            'emotion_classes': emotion_classes,
            'subject_id': subject_id
        }
    
    def _labels_to_classes(self, labels: np.ndarray) -> np.ndarray:
        """Convert valence-arousal to 4 emotion classes"""
        valence = labels[:, 0]
        arousal = labels[:, 1]
        
        emotion_classes = np.zeros(len(labels), dtype=int)
        
        # Q1: High V, High A
        emotion_classes[(valence >= self.config.VALENCE_THRESHOLD) & 
                       (arousal >= self.config.AROUSAL_THRESHOLD)] = 0
        # Q2: Low V, High A
        emotion_classes[(valence < self.config.VALENCE_THRESHOLD) & 
                       (arousal >= self.config.AROUSAL_THRESHOLD)] = 1
        # Q3: Low V, Low A
        emotion_classes[(valence < self.config.VALENCE_THRESHOLD) & 
                       (arousal < self.config.AROUSAL_THRESHOLD)] = 2
        # Q4: High V, Low A
        emotion_classes[(valence >= self.config.VALENCE_THRESHOLD) & 
                       (arousal < self.config.AROUSAL_THRESHOLD)] = 3
        
        return emotion_classes

# Test data loader
print("Testing DEAP Data Loader...")
try:
    loader = DEAPDataLoader()
    test_data = loader.load_subject_data(1)
    print(f"    Loaded subject 1:")
    print(f"  EEG shape: {test_data['eeg_data'].shape}")
    print(f"  Labels shape: {test_data['labels'].shape}")
    print(f"  Emotion classes: {np.unique(test_data['emotion_classes'], return_counts=True)}")
except Exception as e:
    print(f"✗ Error: {e}")
    print(f"  Please ensure DEAP dataset is uploaded and DEAP_PATH is correct")

## 4. Effective Connectivity Measures

In [ ]:
class TransferEntropy:
    """Transfer Entropy with GPU acceleration"""
    
    def __init__(self, num_neighbors=4, embedding_dim=3, time_delay=10, use_gpu=True):
        self.num_neighbors = num_neighbors
        self.embedding_dim = embedding_dim
        self.time_delay = time_delay
        self.use_gpu = use_gpu and GPU_AVAILABLE
        self.xp = cp if self.use_gpu else np
        
    def _embed_signal(self, x, dim, tau):
        """Time-delay embedding"""
        N = len(x)
        M = N - (dim - 1) * tau
        embedded = self.xp.zeros((M, dim))
        for i in range(dim):
            embedded[:, i] = x[i * tau:i * tau + M]
        return embedded
    
    def _knn_entropy(self, x, k):
        """K-NN entropy estimation"""
        N, d = x.shape
        x_cpu = cp.asnumpy(x) if isinstance(x, cp.ndarray) else x
        nbrs = NearestNeighbors(n_neighbors=k + 1, algorithm='auto').fit(x_cpu)
        distances, _ = nbrs.kneighbors(x_cpu)
        rk = distances[:, k]
        rk = self.xp.asarray(rk)
        entropy = d * self.xp.mean(self.xp.log(rk + 1e-10)) + self.xp.log(N) + np.euler_gamma
        return float(entropy)
    
    def compute(self, x: np.ndarray, y: np.ndarray) -> float:
        """Compute TE from x to y"""
        if self.use_gpu:
            x = cp.asarray(x)
            y = cp.asarray(y)
        
        try:
            x_embed = self._embed_signal(x, self.embedding_dim, self.time_delay)
            y_embed = self._embed_signal(y, self.embedding_dim, self.time_delay)
            
            embed_start = (self.embedding_dim - 1) * self.time_delay
            min_len = min(len(x_embed), len(y_embed), len(y) - embed_start - 1)
            
            x_embed = x_embed[:min_len]
            y_embed = y_embed[:min_len]
            y_future = y[embed_start + 1:embed_start + 1 + min_len].reshape(-1, 1)
            
            y_cond = self.xp.hstack([y_embed, y_future])
            xy_cond = self.xp.hstack([y_embed, x_embed, y_future])
            
            h_y = self._knn_entropy(y_cond, self.num_neighbors)
            h_xy = self._knn_entropy(xy_cond, self.num_neighbors)
            
            te = max(0, h_y - h_xy)
            return te
        except:
            return 0.0
    
    def compute_matrix(self, signals: np.ndarray) -> np.ndarray:
        """Compute TE matrix for all channel pairs"""
        num_channels = signals.shape[0]
        te_matrix = np.zeros((num_channels, num_channels))
        
        for i in range(num_channels):
            for j in range(num_channels):
                if i != j:
                    te_matrix[i, j] = self.compute(signals[j], signals[i])
        
        return te_matrix


class PartialDirectedCoherence:
    """PDC with GPU acceleration"""
    
    def __init__(self, model_order=10, use_gpu=True):
        self.model_order = model_order
        self.use_gpu = use_gpu and GPU_AVAILABLE
        self.xp = cp if self.use_gpu else np
    
    def compute_matrix(self, signals: np.ndarray) -> np.ndarray:
        """Compute PDC matrix"""
        num_channels = signals.shape[0]
        pdc_matrix = np.zeros((num_channels, num_channels))
        
        try:
            # Fit VAR model
            from statsmodels.tsa.api import VAR
            model = VAR(signals.T)
            results = model.fit(maxlags=self.model_order, verbose=False)
            
            # Get coefficients
            A = results.params.T
            
            # Compute PDC (simplified)
            for i in range(num_channels):
                for j in range(num_channels):
                    pdc_matrix[i, j] = np.abs(A[i, j]) if i != j else 0
        except:
            # Fallback: use correlation
            corr = np.corrcoef(signals)
            pdc_matrix = np.abs(corr)
            np.fill_diagonal(pdc_matrix, 0)
        
        return pdc_matrix


class DirectDirectedTransferFunction:
    """dDTF with GPU acceleration"""
    
    def __init__(self, model_order=10, use_gpu=True):
        self.model_order = model_order
        self.use_gpu = use_gpu and GPU_AVAILABLE
        self.xp = cp if self.use_gpu else np
    
    def compute_matrix(self, signals: np.ndarray) -> np.ndarray:
        """Compute dDTF matrix"""
        num_channels = signals.shape[0]
        ddtf_matrix = np.zeros((num_channels, num_channels))
        
        try:
            # Similar to PDC but with different normalization
            from statsmodels.tsa.api import VAR
            model = VAR(signals.T)
            results = model.fit(maxlags=self.model_order, verbose=False)
            
            A = results.params.T
            
            for i in range(num_channels):
                for j in range(num_channels):
                    ddtf_matrix[i, j] = np.abs(A[i, j]) if i != j else 0
        except:
            # Fallback
            corr = np.corrcoef(signals)
            ddtf_matrix = np.abs(corr)
            np.fill_diagonal(ddtf_matrix, 0)
        
        return ddtf_matrix


# Install statsmodels for VAR modeling
!pip install -q statsmodels

print("    Connectivity measures initialized")

## 5. Fused Image Generation

In [ ]:
class FusedImageGenerator:
    """Generate fused connectivity images"""
    
    def __init__(self):
        self.config = config
        self.te_estimator = TransferEntropy(use_gpu=config.USE_GPU_CONNECTIVITY)
        self.pdc_estimator = PartialDirectedCoherence(use_gpu=config.USE_GPU_CONNECTIVITY)
        self.ddtf_estimator = DirectDirectedTransferFunction(use_gpu=config.USE_GPU_CONNECTIVITY)
    
    def create_windows(self, eeg_data: np.ndarray) -> List[np.ndarray]:
        """Create overlapping windows"""
        num_channels, num_samples = eeg_data.shape
        window_samples = int(self.config.WINDOW_LENGTH * self.config.SAMPLING_RATE)
        step_samples = int(window_samples * (1 - self.config.OVERLAP_PERCENT))
        
        windows = []
        start = 0
        while start + window_samples <= num_samples:
            windows.append(eeg_data[:, start:start + window_samples])
            start += step_samples
        
        return windows
    
    def normalize_matrix(self, matrix: np.ndarray) -> np.ndarray:
        """Normalize to [0, 255]"""
        if matrix.max() == matrix.min():
            return np.zeros_like(matrix, dtype=np.uint8)
        normalized = (matrix - matrix.min()) / (matrix.max() - matrix.min())
        return (normalized * 255).astype(np.uint8)
    
    def compute_ec_matrices(self, window: np.ndarray) -> Dict[str, np.ndarray]:
        """Compute all EC measures"""
        return {
            'te': self.te_estimator.compute_matrix(window),
            'pdc': self.pdc_estimator.compute_matrix(window),
            'ddtf': self.ddtf_estimator.compute_matrix(window)
        }
    
    def create_fused_image(self, ec_matrices_list: List[Dict]) -> np.ndarray:
        """Create 96x96 fused image from 3 consecutive windows"""
        matrix_size = self.config.EC_IMAGE_SIZE
        num_windows = len(ec_matrices_list)
        
        fused_image = np.zeros((matrix_size * 3, matrix_size * num_windows), dtype=np.uint8)
        
        for win_idx, ec_matrices in enumerate(ec_matrices_list):
            col_start = win_idx * matrix_size
            col_end = (win_idx + 1) * matrix_size
            
            # Stack vertically: dDTF, PDC, TE
            fused_image[0:matrix_size, col_start:col_end] = self.normalize_matrix(ec_matrices['ddtf'])
            fused_image[matrix_size:2*matrix_size, col_start:col_end] = self.normalize_matrix(ec_matrices['pdc'])
            fused_image[2*matrix_size:3*matrix_size, col_start:col_end] = self.normalize_matrix(ec_matrices['te'])
        
        return fused_image
    
    def process_video(self, eeg_data: np.ndarray, video_idx=0, subject_idx=0) -> List[np.ndarray]:
        """Process complete video"""
        windows = self.create_windows(eeg_data)
        
        # Compute EC for all windows
        ec_matrices_all = []
        for window in windows:
            ec_matrices_all.append(self.compute_ec_matrices(window))
        
        # Create fused images from consecutive windows
        fused_images = []
        for i in range(0, len(ec_matrices_all) - self.config.NUM_CONSECUTIVE_WINDOWS + 1):
            ec_subset = ec_matrices_all[i:i + self.config.NUM_CONSECUTIVE_WINDOWS]
            fused_image = self.create_fused_image(ec_subset)
            fused_images.append(fused_image)
        
        return fused_images

print("    Fused image generator initialized")

## 6. PyTorch Dataset

In [ ]:
class FusedImageDataset(Dataset):
    """PyTorch Dataset for fused images"""
    
    def __init__(self, images, labels, target_size=(224, 224)):
        self.images = images
        self.labels = labels
        self.target_size = target_size
        
        self.transform = transforms.Compose([
            transforms.Resize(target_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        # Convert grayscale to RGB
        image = self.images[idx]
        image_rgb = np.stack([image, image, image], axis=-1)
        image_pil = Image.fromarray(image_rgb.astype(np.uint8))
        
        image_tensor = self.transform(image_pil)
        label = self.labels[idx]
        
        return image_tensor, label

print("    PyTorch dataset defined")

## 7. Model Definition with Multi-GPU Support

In [ ]:
class EmotionCNN(nn.Module):
    """Pre-trained CNN for emotion classification"""
    
    def __init__(self, model_name='ResNet50', num_classes=4, pretrained=True):
        super(EmotionCNN, self).__init__()
        self.model_name = model_name
        
        if model_name == 'ResNet50':
            self.base_model = models.resnet50(pretrained=pretrained)
            num_features = self.base_model.fc.in_features
            self.base_model.fc = nn.Linear(num_features, num_classes)
            
        elif model_name == 'InceptionV3':
            self.base_model = models.inception_v3(pretrained=pretrained)
            num_features = self.base_model.fc.in_features
            self.base_model.fc = nn.Linear(num_features, num_classes)
            
        elif model_name == 'DenseNet201':
            self.base_model = models.densenet201(pretrained=pretrained)
            num_features = self.base_model.classifier.in_features
            self.base_model.classifier = nn.Linear(num_features, num_classes)
            
        elif model_name == 'EfficientNetB0':
            self.base_model = models.efficientnet_b0(pretrained=pretrained)
            num_features = self.base_model.classifier[1].in_features
            self.base_model.classifier[1] = nn.Linear(num_features, num_classes)
            
        else:
            raise ValueError(f"Unknown model: {model_name}")
    
    def forward(self, x):
        return self.base_model(x)


def create_model(model_name='ResNet50', num_classes=4, use_multi_gpu=True):
    """Create model with optional multi-GPU support"""
    model = EmotionCNN(model_name, num_classes, pretrained=True)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    # Enable multi-GPU if available
    if use_multi_gpu and torch.cuda.device_count() > 1:
        print(f"Using DataParallel with {torch.cuda.device_count()} GPUs")
        model = nn.DataParallel(model)
    
    return model, device

print("    Model architecture defined")

## 8. Training Functions

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(train_loader, desc='Training'):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc


def evaluate(model, val_loader, criterion, device):
    """Evaluate model"""
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc='Validating'):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    val_loss = running_loss / len(val_loader)
    val_acc = 100. * accuracy_score(all_labels, all_preds)
    
    # Compute metrics
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average='weighted', zero_division=0
    )
    cm = confusion_matrix(all_labels, all_preds)
    
    return val_loss, val_acc, precision, recall, f1, cm

print("    Training functions defined")

## 9. Generate Fused Images for All Subjects

In [ ]:
def generate_all_fused_images(max_subjects=None):
    """Generate fused images for all subjects (or subset for testing)"""
    
    print("\n" + "="*70)
    print("GENERATING FUSED CONNECTIVITY IMAGES")
    print("="*70)
    
    loader = DEAPDataLoader()
    generator = FusedImageGenerator()
    
    all_images = []
    all_labels = []
    
    num_subjects = max_subjects if max_subjects else config.NUM_SUBJECTS
    
    for subject_id in range(1, num_subjects + 1):
        print(f"\nProcessing Subject {subject_id}/{num_subjects}")
        
        try:
            # Load subject data
            subject_data = loader.load_subject_data(subject_id)
            eeg_data = subject_data['eeg_data']
            emotion_classes = subject_data['emotion_classes']
            
            subject_images = []
            subject_labels = []
            
            # Process each video
            for video_idx in tqdm(range(config.NUM_VIDEOS), desc=f"  Videos"):
                video_eeg = eeg_data[video_idx]
                emotion_class = emotion_classes[video_idx]
                
                # Generate fused images
                fused_images = generator.process_video(
                    video_eeg, video_idx, subject_id
                )
                
                subject_images.extend(fused_images)
                subject_labels.extend([emotion_class] * len(fused_images))
            
            all_images.append(subject_images)
            all_labels.append(subject_labels)
            
            print(f"  Generated {len(subject_images)} images for subject {subject_id}")
            
        except Exception as e:
            print(f"  Error processing subject {subject_id}: {e}")
            continue
    
    print(f"\n{'='*70}")
    print(f"GENERATION COMPLETE")
    print(f"  Total subjects: {len(all_images)}")
    print(f"  Total images: {sum(len(imgs) for imgs in all_images):,}")
    print(f"{'='*70}\n")
    
    return all_images, all_labels

# Generate images for subset (testing)
# Use max_subjects=2 for quick testing, or None for all 32 subjects
all_images, all_labels = generate_all_fused_images(max_subjects=2)  # Change to None for full dataset

## 10. LOSO Cross-Validation Training

In [ ]:
def train_loso(all_images, all_labels, model_name='ResNet50', max_epochs=10):
    """LOSO cross-validation with multi-GPU support"""
    
    print("\n" + "="*70)
    print("LOSO CROSS-VALIDATION TRAINING")
    print("="*70)
    print(f"Model: {model_name}")
    print(f"Multi-GPU: {config.USE_MULTI_GPU} ({torch.cuda.device_count()} GPUs)")
    print(f"Max Epochs: {max_epochs}")
    print("="*70)
    
    num_subjects = len(all_images)
    all_results = []
    
    for test_subject_idx in range(num_subjects):
        print(f"\n{'='*70}")
        print(f"FOLD {test_subject_idx + 1}/{num_subjects}")
        print(f"{'='*70}")
        
        # Prepare train/test split
        train_images = []
        train_labels = []
        test_images = all_images[test_subject_idx]
        test_labels = all_labels[test_subject_idx]
        
        for i in range(num_subjects):
            if i != test_subject_idx:
                train_images.extend(all_images[i])
                train_labels.extend(all_labels[i])
        
        print(f"Train samples: {len(train_images):,}")
        print(f"Test samples: {len(test_images):,}")
        
        # Create datasets
        train_dataset = FusedImageDataset(train_images, train_labels)
        test_dataset = FusedImageDataset(test_images, test_labels)
        
        train_loader = DataLoader(
            train_dataset, 
            batch_size=config.BATCH_SIZE,
            shuffle=True,
            num_workers=config.NUM_WORKERS,
            pin_memory=True
        )
        test_loader = DataLoader(
            test_dataset,
            batch_size=config.BATCH_SIZE,
            shuffle=False,
            num_workers=config.NUM_WORKERS,
            pin_memory=True
        )
        
        # Create model with multi-GPU support
        model, device = create_model(model_name, config.NUM_CLASSES, config.USE_MULTI_GPU)
        
        # Loss and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=config.LEARNING_RATE)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3)
        
        # Training loop
        best_acc = 0.0
        history = {'train_loss': [], 'train_acc': [], 'val_acc': []}
        
        for epoch in range(max_epochs):
            print(f"\nEpoch {epoch + 1}/{max_epochs}")
            
            # Train
            train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
            
            # Validate
            val_loss, val_acc, precision, recall, f1, cm = evaluate(
                model, test_loader, criterion, device
            )
            
            # Update scheduler
            scheduler.step(val_acc)
            
            # Save history
            history['train_loss'].append(train_loss)
            history['train_acc'].append(train_acc)
            history['val_acc'].append(val_acc)
            
            print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
            print(f"  Val Acc: {val_acc:.2f}%, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
            
            # Save best model
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), 
                          os.path.join(MODEL_DIR, f'{model_name}_fold{test_subject_idx}_best.pth'))
        
        # Save fold results
        fold_results = {
            'fold': test_subject_idx,
            'best_acc': best_acc,
            'final_acc': val_acc,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'confusion_matrix': cm.tolist(),
            'history': history
        }
        all_results.append(fold_results)
        
        print(f"\n  Fold {test_subject_idx + 1} Best Accuracy: {best_acc:.2f}%")
    
    # Overall statistics
    print(f"\n{'='*70}")
    print("FINAL LOSO RESULTS")
    print(f"{'='*70}")
    
    best_accs = [r['best_acc'] for r in all_results]
    mean_acc = np.mean(best_accs)
    std_acc = np.std(best_accs)
    
    print(f"Mean Accuracy: {mean_acc:.2f}% ± {std_acc:.2f}%")
    print(f"Min Accuracy: {min(best_accs):.2f}%")
    print(f"Max Accuracy: {max(best_accs):.2f}%")
    
    # Save results
    with open(os.path.join(RESULTS_DIR, f'{model_name}_loso_results.json'), 'w') as f:
        json.dump({
            'mean_accuracy': mean_acc,
            'std_accuracy': std_acc,
            'all_folds': all_results
        }, f, indent=2)
    
    return all_results

# Train with LOSO
results = train_loso(all_images, all_labels, model_name=config.CNN_MODEL, max_epochs=5)  # Use 40 for full training

## 11. Visualization

In [ ]:
# Plot training history
def plot_results(results, model_name):
    """Plot LOSO results"""
    
    # Accuracy per fold
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Fold accuracies
    fold_ids = [r['fold'] + 1 for r in results]
    best_accs = [r['best_acc'] for r in results]
    
    axes[0].bar(fold_ids, best_accs)
    axes[0].axhline(np.mean(best_accs), color='r', linestyle='--', label=f'Mean: {np.mean(best_accs):.2f}%')
    axes[0].set_xlabel('Fold (Test Subject)')
    axes[0].set_ylabel('Accuracy (%)')
    axes[0].set_title(f'{model_name} - LOSO Accuracy per Fold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Training curves (first fold as example)
    history = results[0]['history']
    epochs = range(1, len(history['train_acc']) + 1)
    
    axes[1].plot(epochs, history['train_acc'], label='Train Acc', marker='o')
    axes[1].plot(epochs, history['val_acc'], label='Val Acc', marker='s')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy (%)')
    axes[1].set_title(f'{model_name} - Training Curve (Fold 1)')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, f'{model_name}_results.png'), dpi=300, bbox_inches='tight')
    plt.show()
    
    # Confusion matrix (average across folds)
    avg_cm = np.mean([np.array(r['confusion_matrix']) for r in results], axis=0)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(avg_cm, annot=True, fmt='.1f', cmap='Blues',
                xticklabels=config.CLASS_NAMES,
                yticklabels=config.CLASS_NAMES)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'{model_name} - Average Confusion Matrix')
    plt.savefig(os.path.join(FIGURES_DIR, f'{model_name}_confusion_matrix.png'), dpi=300, bbox_inches='tight')
    plt.show()

# Plot results
plot_results(results, config.CNN_MODEL)

## 12. Save and Download Results

In [ ]:
# Create summary report
summary = {
    'model': config.CNN_MODEL,
    'dataset': 'DEAP',
    'num_subjects': len(all_images),
    'total_images': sum(len(imgs) for imgs in all_images),
    'mean_accuracy': np.mean([r['best_acc'] for r in results]),
    'std_accuracy': np.std([r['best_acc'] for r in results]),
    'gpu_count': torch.cuda.device_count(),
    'multi_gpu_enabled': config.USE_MULTI_GPU,
    'connectivity_gpu_enabled': config.USE_GPU_CONNECTIVITY,
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

with open(os.path.join(RESULTS_DIR, 'summary.json'), 'w') as f:
    json.dump(summary, f, indent=2)

print("\n" + "="*70)
print("TRAINING SUMMARY")
print("="*70)
for key, value in summary.items():
    print(f"{key}: {value}")
print("="*70)

print(f"\n    All results saved to: {OUTPUT_DIR}")
print(f"  - Models: {MODEL_DIR}")
print(f"  - Results: {RESULTS_DIR}")
print(f"  - Figures: {FIGURES_DIR}")